In [ ]:
#pip install --user pandas

In [ ]:
import pandas as pd

In [ ]:
#pip install --user scikit-learn

In [ ]:
from sklearn.datasets import load_boston

In [ ]:
l = load_boston()

In [ ]:
l.feature_names

In [ ]:
l.data

In [ ]:
l.target

In [ ]:
print(l.DESCR)

In [ ]:
df = pd.DataFrame(l.data, columns=l.feature_names)

In [ ]:
df['PRICE'] = l.target

In [ ]:
df.head()

In [ ]:
# Understand the data --> Data dictionary

In [ ]:
# Explore, clean & manipulate the data as required

In [ ]:
df.shape

In [ ]:
round(df.describe(),2).T

In [ ]:
#!pip install --user seaborn

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(y=df['PRICE'])

In [ ]:
#!pip install --user matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(df['CRIM'],df['PRICE'])
plt.xlabel("CRIME")
plt.ylabel("PRICE")
plt.show()

In [ ]:
plt.scatter(df['RM'],df['PRICE'])
plt.xlabel("RM")
plt.ylabel("PRICE")
plt.show()

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
# Linear Regression

In [ ]:
# 1st assumption --> All X vars should have a correlation with Y

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(df.corr(),annot=True)

In [ ]:
# Check the distribution of the Target variable

In [ ]:
sns.distplot(df['PRICE'])

In [ ]:
sns.boxplot(y=df['PRICE'])

In [ ]:
x = df.iloc[:,:13]

In [ ]:
y = df['PRICE']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.20, random_state=2)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_train[:5]

In [ ]:
y_train[:5]

In [ ]:
sns.boxplot(y=y_train)

In [ ]:
sns.boxplot(y=y_test)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
model = lr.fit(x_train,y_train)

In [ ]:
model

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
pred_price = model.predict(x_test)

In [ ]:
pred_price[:5]

In [ ]:
y_test[:5]

In [ ]:
x_test[:1]

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(y_test,pred_price)

In [ ]:
mse

In [ ]:
import numpy as np

In [ ]:
np.sqrt(mean_squared_error(y_test,pred_price))

In [ ]:
err = pd.DataFrame({'Actual':y_test,"Predicted":pred_price})

In [ ]:
err = err.reset_index(drop=True)

In [ ]:
plt.plot(err)

In [ ]:
import findspark

In [ ]:
findspark.init('/usr/local/spark')

In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('ML').getOrCreate()

In [ ]:
df.to_csv('boston1.csv')

In [ ]:
# Build the model in Pyspark

In [ ]:
df1 = spark.read.csv('boston1.csv', header=True, inferSchema=True)

In [ ]:
df1.printSchema()

In [ ]:
df1=df1.drop('_c0')

In [ ]:
df1.columns

In [ ]:
# df2 = df1.toPandas()

In [ ]:
# Spark MLlib --> for Spark RDD syntax
# Spark ML --> for Spark ML

In [ ]:
# Data transformation

In [ ]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [ ]:
df1.columns

In [ ]:
assmblr = VectorAssembler(inputCols=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],outputCol="features")

In [ ]:
output = assmblr.transform(df1)

In [ ]:
final_df = output.select("features","PRICE")

In [ ]:
df1.show(5)

In [ ]:
final_df.show(5)

In [ ]:
train_data,test_data = final_df.randomSplit([0.8,0.2])

In [ ]:
train_data.show(5)

In [ ]:
test_data.show(5)

In [ ]:
train_data.count()

In [ ]:
test_data.count()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(labelCol='PRICE')

In [ ]:
model = lr.fit(train_data)

In [ ]:
model.coefficients

In [ ]:
x.columns

In [ ]:
model.intercept

In [ ]:
pred_price1 = model.evaluate(test_data)

In [ ]:
pred_price1.predictions.show()

In [ ]:
pred_price1.rootMeanSquaredError